<a href="https://colab.research.google.com/github/TSA-bit/big-data-challenge/blob/main/Level1_wireless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
#connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-07 03:27:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.34MB/s    in 0.2s    

2022-03-07 03:27:00 (5.34 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
#start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazon_reviews_us_Wireless").config("apark.driver.extraClassPath", "/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
#Read in data from S3 buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep="\t")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [8]:
#check datatypes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
#change review_date to date format
from pyspark.sql.types import DateType

df = df.withColumn("review_date", df.review_date.cast(DateType()))
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [10]:
#drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

9002021
9001052
9001052


In [11]:
#verify data types after changes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
#create review_id_table
reviews_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1003OGUXVB9L7|   16562564|B00ENOBLPM|     169468004| 2013-12-11|
|R1006P92KP8KA8|   51045659|B009PDKFBW|     128785928| 2012-12-08|
|R1006ZF2AO1MDF|   42032306|B00KR4CUCU|     645294944| 2014-08-17|
|R1007XR39U7P38|   39378358|B00QLSE0P6|     334027678| 2015-07-08|
|R10092JBH5HHQQ|    2010712|B011R5O6YC|     609891243| 2015-08-31|
|R100NQRQP6QC2D|   27891243|B00HUDMVFC|     536452055| 2015-03-25|
|R100W4K4320LX3|   44564552|B00ADI2ACA|     345697578| 2014-06-17|
|R1010WELCBMCNW|   15694682|B0051SKEKW|     529701170| 2014-12-09|
|R10170S8Z8SZ8M|   25812141|B00KB8298C|     277778602| 2014-12-07|
|R1017CEDHQJ2J5|   16441985|B00FW5NALE|     513044566| 2014-05-12|
|R101BQXSQBJ1XV|     948792|B00SM1TJ0Q|     100625298| 2015-05-26|
|R101NTXIVGQ66M|    2143789|B00E486Q48|     354195694| 2013-11

In [13]:
#create product table
product_df = df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00ENOBLPM|[ iPhone 5C Case]...|
|B009PDKFBW|[Buy World] for M...|
|B00KR4CUCU|COVER U (TM) Sams...|
|B00QLSE0P6|URPOWER® Portable...|
|B011R5O6YC|iPhone 6 PLUS Arm...|
|B00HUDMVFC|DESIGNCOOL Outlet...|
|B00ADI2ACA|TRIDENT Kraken AM...|
|B0051SKEKW|Samsung Infuse 4G...|
|B00KB8298C|MagicShield Color...|
|B00FW5NALE|[2 Pack] Supershi...|
|B00SM1TJ0Q|Samsung Galaxy Co...|
|B00E486Q48|Spigen Ultra Fit ...|
|B00I9I2NOG|Griffin Vehicle D...|
|B00N1V5WLM|iPhone 6 Case, ES...|
|B004V40RKY|Generic HTC Incre...|
|B00BTGNS3A|GENERIC Stereo Ea...|
|B00WV7AMN0|Bluetooth Headpho...|
|B003ADBQN4|Replacement Apple...|
|B008RZ0EQ0|Baofeng BF-S112 T...|
|B009QZH49K|Samsung Galaxy S ...|
+----------+--------------------+
only showing top 20 rows



In [14]:
#count total number rows in product_df
counted = product_df.count()
print(f"Total number product: {counted}")

Total number product: 9001052


In [15]:
#product table needs only unique values
product_df = product_df.distinct()

#check new row count
recounted = product_df.count()
print(f"Total number after recount: {recounted}")

Total number after recount: 906482


In [16]:
#create customers table, per schema need customer count
customers_df = df.groupBy("customer_id").count()

#rename count column
customers_df = customers_df.withColumnRenamed("count", "customer_count")

#order by customer count
customers_df.orderBy(customers_df["customer_count"].desc()).show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30208851|           616|
|   53037408|           493|
|   45070473|           437|
|   49266466|           435|
|   32038204|           427|
|   52870270|           353|
|   44834233|           330|
|   11995502|           319|
|   51346302|           318|
|   43856165|           316|
|   15886030|           292|
|   35353346|           284|
|   12200139|           267|
|   50027179|           266|
|   52228204|           262|
|   13355404|           261|
|   53090839|           261|
|   52496677|           259|
|   51712970|           255|
|   39789300|           242|
+-----------+--------------+
only showing top 20 rows



In [17]:
#check data type for customers_df
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [18]:
#per schema customer count needs to be integer
from pyspark.sql.types import IntegerType

customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))

#verify data type change
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [19]:
#Create vine_table
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1003OGUXVB9L7|          2|            0|          1|   N|
|R1006P92KP8KA8|          4|            0|          0|   N|
|R1006ZF2AO1MDF|          5|            0|          0|   N|
|R1007XR39U7P38|          1|            0|          0|   N|
|R10092JBH5HHQQ|          5|            0|          1|   N|
|R100NQRQP6QC2D|          5|            0|          0|   N|
|R100W4K4320LX3|          3|            0|          0|   N|
|R1010WELCBMCNW|          5|            0|          0|   N|
|R10170S8Z8SZ8M|          1|            5|          7|   N|
|R1017CEDHQJ2J5|          5|            0|          0|   N|
|R101BQXSQBJ1XV|          5|            0|          0|   N|
|R101NTXIVGQ66M|          4|            0|          0|   N|
|R101OME37K3K0Z|          1|            0|          0|   N|
|R101QYK6Y5BXT1|          1|            

In [20]:
#configure settings for RDS
mode = "append"
jdbc_url = "jdbc:postgresql://<connection string>:5432/<luggage>"
config = {"user":"<USERNAME>", 
          "password": "<PASSWORD>", 
          "driver":"org.postgresql.Driver"}


In [ ]:
#write dataframe to review_id table in RDS
reviews_df.write.jdbc(url=jdbc_url, table="review_id", mode=mode, properties=config)

In [ ]:
#write dataframe to product table in RDS
product_df.write.jdbc(url=jdbc_url, table="product", mode=mode, properties=config)

In [ ]:
#write dataframe to customers in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
#write dataframe to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)